# Распознавание лиц для видео
Изображения обнаруженных лиц имеют формат `frameXfaceY.jpg`,
где `X` представляет X кадр, а `Y` - Y лицо в X-м кадре.

In [ ]:
import os
from pathlib import Path

import cv2
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.layers import Input
from matplotlib import pyplot as plt
from moviepy.editor import VideoFileClip

In [ ]:
from umeyama import umeyama
from detector import mtcnn

In [ ]:
RATIO_LANDMARKS = [
    (0.31339227236234224, 0.3259269274198092),
    (0.31075140146108776, 0.7228453709528997),
    (0.5523683107816256, 0.5187296867370605),
    (0.7752419985257663, 0.37262483743520886),
    (0.7759613623985877, 0.6772957581740159)
]

Создайте MTCNN и его функции forward pass

In [ ]:
WEIGHTS_PATH = "./mtcnn_weights/"

In [ ]:
def create_mtcnn(session, model_path):
    if not model_path:
        model_path, _ = os.path.split(os.path.realpath(__file__))

    with tf.variable_scope('pnet2'):
        data = Input((None, None, None, 3), dtype=tf.float32, name='input')
        pnet_ = mtcnn.PNet({'data': data})
        pnet_.load(os.path.join(model_path, 'det1.npy'), session)
    with tf.variable_scope('rnet2'):
        data = Input((None, 24, 24, 3), tf.float32, 'input')
        rnet_ = mtcnn.RNet({'data': data})
        rnet_.load(os.path.join(model_path, 'det2.npy'), session)
    with tf.variable_scope('onet2'):
        data = Input((None, 48, 48, 3), tf.float32, 'input')
        onet_ = mtcnn.ONet({'data': data})
        onet_.load(os.path.join(model_path, 'det3.npy'), session)
    return pnet_, rnet_, onet_

In [ ]:
sess = K.get_session()
with sess.as_default():
    global pnet, rnet, onet
    pnet, rnet, onet = create_mtcnn(sess, WEIGHTS_PATH)
    pass

global pnet, rnet, onet

pnet = K.function(
    [pnet.layers['data']],
    [pnet.layers['conv4-2'], pnet.layers['prob1']]
)
rnet = K.function(
    [rnet.layers['data']],
    [rnet.layers['conv5-2'], rnet.layers['prob1']]
)
onet = K.function(
    [onet.layers['data']],
    [onet.layers['conv6-2'], onet.layers['conv6-3'], onet.layers['prob1']]
)

Создайте папку, в которую будут сохраняться изображения

Устанавливаем глобальные константы для путей сохранения

In [ ]:
FACES_PATH="/content/drive/MyDrive/faces"

DIR_FACES_ALGN = f"{FACES_PATH}/aligned_faces"
DIR_FACES_RAW=f"{FACES_PATH}/raw_faces"
DIR_FACES_BME=f"{FACES_PATH}/binary_masks_eyes"

Path(DIR_FACES_ALGN).mkdir(parents=True, exist_ok=True)
Path(DIR_FACES_RAW).mkdir(parents=True, exist_ok=True)
Path(DIR_FACES_BME).mkdir(parents=True, exist_ok=True)

Функции для обработки видео и выравнивания лиц

In [ ]:
def get_src_landmarks(x0, x1, y0, y1, pnts):
    """
    x0, x1, y0, y1: (smoothed) bbox coord.
    pnts: landmarks predicted by MTCNN
    """
    src_landmarks = [(int(pnts[i + 5][0] - x0),
                      int(pnts[i][0] - y0)) for i in range(5)]
    return src_landmarks


def get_tar_landmarks(img):
    """    
    img: detected face image
    """
    img_size = img.shape
    tar_landmarks = [(int(xy[0] * img_size[0]),
                      int(xy[1] * img_size[1])) for xy in RATIO_LANDMARKS]
    return tar_landmarks


def landmarks_match_mtcnn(src_im, src_landmarks, tar_landmarks):
    """
    umeyama(src, dst, estimate_scale)
    landmarks coord. for umeyama should be (width, height) or (y, x)
    """
    src_size = src_im.shape
    src_tmp = [(int(xy[1]), int(xy[0])) for xy in src_landmarks]
    tar_tmp = [(int(xy[1]), int(xy[0])) for xy in tar_landmarks]
    M = umeyama(np.array(src_tmp), np.array(tar_tmp), True)[0:2]
    result = cv2.warpAffine(src_im, M, (src_size[1], src_size[0]), borderMode=cv2.BORDER_REPLICATE)
    return result


def process_mtcnn_bbox(bboxes, im_shape):
    """
    output bbox coordinate of MTCNN is (y0, x0, y1, x1)
    Here we process the bbox coord. to a square bbox with ordering (x0, y1, x1, y0)
    """
    for i, bbox in enumerate(bboxes):
        y0, x0, y1, x1 = bboxes[i, 0:4]
        w, h = int(y1 - y0), int(x1 - x0)
        length = (w + h) / 2
        center = (int((x1 + x0) / 2), int((y1 + y0) / 2))
        new_x0 = np.max([0, (center[0] - length // 2)])  #.astype(np.int32)
        new_x1 = np.min([im_shape[0], (center[0] + length // 2)])  #.astype(np.int32)
        new_y0 = np.max([0, (center[1] - length // 2)])  #.astype(np.int32)
        new_y1 = np.min([im_shape[1], (center[1] + length // 2)])  #.astype(np.int32)
        bboxes[i, 0:4] = new_x0, new_y1, new_x1, new_y0
        pass
    return bboxes


def process_video(input_img):
    global frames, save_interval
    global pnet, rnet, onet
    minsize = 30  # minimum size of face
    detec_threshold = 0.7
    threshold = [0.6, 0.7, detec_threshold]  # three steps's threshold
    factor = 0.709  # scale factor

    frames += 1
    if frames % save_interval == 0:
        faces, pnts = mtcnn.detect_face(
            input_img, minsize, pnet, rnet, onet, threshold, factor
        )
        faces = process_mtcnn_bbox(faces, input_img.shape)

        for idx, (x0, y1, x1, y0, conf_score) in enumerate(faces):
            det_face_im = input_img[int(x0):int(x1), int(y0):int(y1), :]

            # get src/tar landmarks
            src_landmarks = get_src_landmarks(x0, x1, y0, y1, pnts)
            tar_landmarks = get_tar_landmarks(det_face_im)

            # align detected face
            aligned_det_face_im = landmarks_match_mtcnn(
                det_face_im, src_landmarks, tar_landmarks)

            fname = f"{DIR_FACES_ALGN}/frame{frames}face{str(idx)}.jpg"
            plt.imsave(fname, aligned_det_face_im, format="jpg")
            fname = f"{DIR_FACES_RAW}/frame{frames}face{str(idx)}.jpg"
            plt.imsave(fname, det_face_im, format="jpg")

            bm = np.zeros_like(aligned_det_face_im)
            h, w = bm.shape[:2]
            bm[int(src_landmarks[0][0] - h / 15):int(src_landmarks[0][0] + h / 15),
            int(src_landmarks[0][1] - w / 8):int(src_landmarks[0][1] + w / 8), :] = 255
            bm[int(src_landmarks[1][0] - h / 15):int(src_landmarks[1][0] + h / 15),
            int(src_landmarks[1][1] - w / 8):int(src_landmarks[1][1] + w / 8), :] = 255
            bm = landmarks_match_mtcnn(bm, src_landmarks, tar_landmarks)
            fname = f"{DIR_FACES_BME}/frame{frames}face{str(idx)}.jpg"
            plt.imsave(fname, bm, format="jpg")
            pass
        pass

    return np.zeros((3, 3, 3))

Начать распознавание лиц

Имя файла входного видео по умолчанию: `source.mp4`

In [ ]:
frames = 0

# configuration
save_interval = 6  # perform face detection every {save_interval} frames
fn_input_video = "samples/source.mp4"

output = 'dummy.mp4'
clip1 = VideoFileClip(fn_input_video)
clip = clip1.fl_image(process_video)  #.subclip(0,3) #NOTE: this function expects color images!!
clip.write_videofile(output, audio=False)
clip1.reader.close()

## Сохраненные изображения будут находиться в папке `faces/raw_faces` и `faces/aligned_faces` соответственно.
## Бинарные маски будут находиться в `faces/binary_masks_eyes`.